<a href="https://colab.research.google.com/github/ProAek11/StrategyTradingFrendersis-AI-System/blob/main/copy_of_livetradingsignal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf

def get_data(symbol="AAPL"):
    # Define the ticker symbol
    ticker_symbol = symbol

    # Create a ticker object
    ticker = yf.Ticker(ticker_symbol)

    # Download historical data
    return ticker.history(period="2d", interval='1d')

In [ ]:
# Download Recent historical Data
historical_data = get_data()
historical_data

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2023-08-17 00:00:00-04:00,177.139999,177.509995,173.479996,174.000000,66062900,0,0
2023-08-18 00:00:00-04:00,172.300003,175.100006,171.960007,174.490005,61114200,0,0


In [ ]:
def test_engulfing(df):
    last_open = df.iloc[-1, :].Open
    last_close = df.iloc[-1, :].Close
    previous_open = df.iloc[-2, :].Open
    previous_close = df.iloc[-2, :].Close

    if (previous_open < previous_close
        and last_open > previous_close
        and last_close < previous_open):
        return 1  # Bearish Engulfing Pattern

    elif (previous_open > previous_close
          and last_open < previous_close
          and last_close > previous_open):
        return 2  # Bullish Engulfing Pattern
    else:
        return 0  # No Engulfing Pattern

In [ ]:
test_engulfing(historical_data)

0

In [ ]:
#_____________________________________
# Test the engulfing function
#°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°

# historical_data.iloc[-1, :].Open
# historical_data.iloc[-1, -4] = 176
# historical_data.iloc[-2, :].Open
# historical_data.iloc[-2, :].Close

test_engulfing(historical_data)
# historical_data

0

# Send Live Signal

In [ ]:
from apscheduler.schedulers.blocking import BlockingScheduler
from email.message import EmailMessage
import ssl
import smtplib
from creds import gmail_user, gmail_password

em = EmailMessage()

gmail_user = gmail_user
gmail_password = gmail_password
subject = 'info signal'


def some_job():
    msg="Trading Signal Message \n"
    historical_data = get_data()

    if test_engulfing(historical_data)==1:
        msg = str("the signal is 1 bearish")

    elif test_engulfing(historical_data)==2:
        msg = str("the signal is 2 bullish")

    em['From'] = gmail_user
    em['To'] = gmail_user
    em['Subject'] = subject
    em.set_content(msg)

    context = ssl.create_default_context()

    server = smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context)
    server.ehlo()
    server.login(gmail_user, gmail_password)
    server.sendmail(gmail_user, gmail_user, em.as_string())
    server.close()

some_job()

###################################################################
## Interval time job ##############################################
# scheduler = BlockingScheduler(job_defaults={'misfire_grace_time': 15*60})
# scheduler.add_job(some_job, 'cron', day_of_week='mon-fri', hour=0, minute=0, timezone=utc)
# scheduler.start()

# Live Signal Multiple Stocks

In [ ]:
from apscheduler.schedulers.blocking import BlockingScheduler
from email.message import EmailMessage
import ssl
import smtplib

from creds import gmail_user, gmail_password

em = EmailMessage()

gmail_user = gmail_user
gmail_password = gmail_password
subject = 'info signal'

symbols =  ['AAPL', 'NVDA', 'PYPL']

def some_job():
    msg="Trading Signal Message \n"
    for symb in symbols:
        historical_data = get_data(symb)
        if test_engulfing(historical_data)==1:
            msg = msg + str(symb+": the signal is 1 bearish") + "\n"

        elif test_engulfing(historical_data)==2:
            msg = msg + str(symb+": the signal is 2 bullish") + "\n"

    em['From'] = gmail_user
    em['To'] = gmail_user
    em['Subject'] = subject
    em.set_content(msg)

    context = ssl.create_default_context()

    server = smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context)
    server.ehlo()
    server.login(gmail_user, gmail_password)
    server.sendmail(gmail_user, gmail_user, em.as_string())
    server.close()

#some_job()

###################################################################
## Interval time job ##############################################
scheduler = BlockingScheduler(job_defaults={'misfire_grace_time': 15*60})
scheduler.add_job(some_job, 'cron', day_of_week='mon-fri', hour=0, minute=0, timezone=utc)
scheduler.start()